# A example of integration with ADSL Gen2

In [0]:
service_principal_application_id = ""
service_principal_secret = ""                            
service_principal_tenant_id = ""

extra_configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": service_principal_application_id,
    "fs.azure.account.oauth2.client.secret": service_principal_secret,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{service_principal_tenant_id}/oauth2/token",
    "fs.azure.createRemoteFileSystemDuringInitialization": "true"
}

container_name = "container1"
storage_account_name = "st24012021"
source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/"

mount_point = "/mnt/netcoreconf"

dbutils.fs.mount(
  source = source,
  mount_point = mount_point,
  extra_configs = extra_configs)

Out[3]: True

In [0]:
%sh ls /dbfs/mnt/netcoreconf -la -R


/dbfs/mnt/netcoreconf:
total 12
drwxrwxrwx 2 root root 4096 Feb 24 13:49 .
drwxrwxrwx 2 root root 4096 Feb 24 13:49 ..
drwxrwxrwx 2 root root 4096 Feb 24 13:49 directory1

/dbfs/mnt/netcoreconf/directory1:
total 12
drwxrwxrwx 2 root root 4096 Feb 24 13:32 .
drwxrwxrwx 2 root root 4096 Feb 24 13:49 ..
drwxrwxrwx 2 root root 4096 Feb 24 13:32 daily_stocks

/dbfs/mnt/netcoreconf/directory1/daily_stocks:
total 11
drwxrwxrwx 2 root root 4096 Feb 24 13:32 .
drwxrwxrwx 2 root root 4096 Feb 24 13:32 ..
-rwxrwxrwx 1 root root 1268 Feb 24 13:32 part-00000-2d772140-8c3b-47ad-a592-9862a0fef762-c000.snappy.parquet
-rwxrwxrwx 1 root root 20 Feb 24 13:32 .part-00000-2d772140-8c3b-47ad-a592-9862a0fef762-c000.snappy.parquet.crc
-rwxrwxrwx 1 root root 0 Feb 24 13:32 _SUCCESS
-rwxrwxrwx 1 root root 8 Feb 24 13:32 ._SUCCESS.crc

In [0]:
spark.read.parquet("/mnt/netcoreconf/directory1/daily_stocks").show()

+-------+---------+-------------------+--------+
StoreId|ProductId| Date|Quantity|
+-------+---------+-------------------+--------+
 1| 1|2019-12-31 23:00:00| 5|
 1| 1|2020-01-01 23:00:00| 5|
 1| 1|2020-01-02 23:00:00| 8|
 1| 1|2020-01-03 23:00:00| 6|
 1| 1|2020-01-04 23:00:00| 1|
 1| 1|2020-01-05 23:00:00| -3|
 1| 1|2020-01-09 23:00:00| 2|
 1| 2|2019-12-31 23:00:00| 3|
 1| 2|2020-01-01 23:00:00| 3|
 1| 2|2020-01-02 23:00:00| 8|
 1| 2|2020-01-03 23:00:00| 8|
 1| 2|2020-01-04 23:00:00| 8|
 1| 2|2020-01-05 23:00:00| 2|
 1| 2|2020-01-06 23:00:00| 1|
 1| 2|2020-01-09 23:00:00| 2|
 2| 1|2019-12-31 23:00:00| 1|
 2| 1|2020-01-01 23:00:00| 1|
 2| 1|2020-01-04 23:00:00| 7|
 2| 1|2020-01-05 23:00:00| 2|
 2| 1|2020-01-09 23:00:00| 1|
+-------+---------+-------------------+--------+
only showing top 20 rows

In [0]:
df = spark.read.parquet("/mnt/netcoreconf/directory1/daily_stocks")
df.createOrReplaceTempView("my_view")

In [0]:
%sql
select * from my_view;

StoreId,ProductId,Date,Quantity
1,1,2019-12-31T23:00:00.000+0000,5
1,1,2020-01-01T23:00:00.000+0000,5
1,1,2020-01-02T23:00:00.000+0000,8
1,1,2020-01-03T23:00:00.000+0000,6
1,1,2020-01-04T23:00:00.000+0000,1
1,1,2020-01-05T23:00:00.000+0000,-3
1,1,2020-01-09T23:00:00.000+0000,2
1,2,2019-12-31T23:00:00.000+0000,3
1,2,2020-01-01T23:00:00.000+0000,3
1,2,2020-01-02T23:00:00.000+0000,8


In [0]:
df.write.saveAsTable("my_table")

In [0]:
mount_point = "/mnt/netcoreconf"
dbutils.fs.unmount(mount_point)

/mnt/netcoreconf has been unmounted.
Out[21]: True